All experimental phosphorylation data available 

Paper Nature

<img src='./img/nature_2020_cantidades_exp.png'></img>

## Initial Setup

In [ ]:
from sys import path
base_path = "/media/paulati/Nuevo vol/paula/ingebi/2020/agustina_mazzella/github/arabidopsis_phospho"
path.append(base_path)    

from preparation import util

## Imports

In [ ]:
from os import path, remove
import pandas as pd
import numpy as np
import re

## Arabidopsis all ids

In [ ]:
all_arabidopsisis_file_path = path.join(base_path, "data/preproc/all_ids.txt")
all_arabidopsisis = pd.read_csv(all_arabidopsisis_file_path, header = None)
print(all_arabidopsisis.shape)
all_arabidopsisis_codes = all_arabidopsisis.iloc[:, 0].unique()
len(all_arabidopsisis_codes)

## Phosphat experimental

In [ ]:
data_path_zip = path.join(base_path, "data/raw/Phosphat_20200624.zip")
data_folder_path = path.join(base_path, "data/raw")
data_path = path.join(data_folder_path, "Phosphat_20200624.csv")

if (not path.exists(data_path)):
    util.extractzip(data_path_zip, data_folder_path)

### Fix data

Problem: `FK_Species` is mixed with other columns.

In [ ]:
phosphat_experimental = pd.read_csv(data_path_zip, sep = ',', quotechar = "\"", low_memory=False)
phosphat_experimental.FK_Species.unique()

In [ ]:
f = open(data_path)
line_number = 1
error_line_number = []

error_pattern = ',,,,,'

for line in f:
    if line.find(error_pattern) != -1:
        error_line_number.append(line_number)
    line_number = line_number + 1

Detect problems in consecutive lines pairs:

In [ ]:
elem_count = len(error_line_number)
max_index = int(elem_count / 2)

print(elem_count)
print(max_index)

sum([error_line_number[2*i]+1 == error_line_number[2*i+1] for i in range(0, max_index)])

In [ ]:
f = open(data_path, 'r')

raw_lines = f.readlines()

error_line_index = [x-1 for x in error_line_number]

original_valid_lines = [raw_lines[i] for i in range(0, len(raw_lines)) if i not in error_line_index]    

phosphat_experimental_2020_valid_data_file_path = path.join(base_path, "data/preproc/Phosphat_20200624_valid_lines.csv")

f_out = open(phosphat_experimental_2020_valid_data_file_path, 'w')
f_out.writelines(original_valid_lines)        
f_out.close()

Read file `Phosphat_20200624_valid_lines` and verify that shape is 24 columns and (162857 - 4436) rows

In [ ]:
tmp = pd.read_csv(phosphat_experimental_2020_valid_data_file_path, sep = ',', quotechar = "\"", low_memory=False)

print(tmp.head(5))
print(tmp.shape[0] ==  162857 - 4436)
print(tmp.shape[1] == 24)

Check problem is solved:

In [ ]:
tmp.FK_Species.unique()

### Extract phosphat experimental ids

In [ ]:
phosphat_experimental_2020_valid_data_file_path = path.join(base_path, "data/preproc/Phosphat_20200624_valid_lines.csv")
phosphat_exp = pd.read_csv(phosphat_experimental_2020_valid_data_file_path, sep = ',', quotechar = "\"", low_memory=False)

phosphat_exp_clean_codes = phosphat_exp.code.apply(lambda x: x.split(".")[0])
phosphat_exp_clean_codes_unique = phosphat_exp_clean_codes.unique()
 
# 8023 in 2016
len(phosphat_exp_clean_codes_unique)

In [ ]:
phosphat_exp_clean_codes_unique

Check if these are valid codes included in all_ids

In [ ]:
valid_exp_ids = [x for x in phosphat_exp_clean_codes_unique if x in all_arabidopsisis_codes]
len(valid_exp_ids)

In [ ]:
result = pd.DataFrame(valid_exp_ids, columns = ['code'])
phosphat_result_2020_file_path = path.join(base_path, 'data/preproc/phosphat_experimental_2020.csv')
result.to_csv(phosphat_result_2020_file_path, index = False)

### Clean tmp files:

In [ ]:
phosphat_result_2020_file_path_zip = phosphat_result_2020_file_path + ".zip"

if(path.exists(phosphat_result_2020_file_path)):
    if(path.exists(phosphat_result_2020_file_path_zip)):
        to_delete = True
    else:
        output_folder_path = path.join(base_path, "data")
        util.compresszip(phosphat_result_2020_file_path, output_folder_path)        
        to_delete = True
else:
    to_delete = False
    
if (to_delete):
    remove(phosphat_result_2020_file_path)
    
        

## p3db experimental

In [ ]:
file_name = 'p3db-3.5-phosphosite-report_Arabidopsis-thaliana.gz'
file_path = path.join(base_path, 'data/raw', file_name)

out_file_name = 'p3db-3.5_experimental.csv'
out_file_path = path.join(base_path, 'data/preproc', out_file_name)

data = pd.read_csv(file_path, sep = '\t')
data.shape

In [ ]:
def parse_tair_id(raw_data):    
    parts = raw_data.split(';')
    tair_mask = [part.find('TAIR') != -1 for part in parts]
    tair_ids = pd.Series(parts).loc[tair_mask]        
    if(len(tair_ids) > 0):
        tair_id = tair_ids.iloc[0]
    else:
        tair_id = None
    result = tair_id.replace('TAIR:', '')
    return(result)

In [ ]:
p3db_tair_ids = data.Xref.apply(lambda x: parse_tair_id(x))
p3db_tair_ids

In [ ]:
p3db_tair_ids_clean = [x.split(".")[0] for x in p3db_tair_ids]

In [ ]:
p3db_tair_ids_valid = [x for x in p3db_tair_ids_clean if x in all_arabidopsisis_codes]
p3db_tair_ids_valid_unique = np.unique(p3db_tair_ids_valid)
print(len(p3db_tair_ids.unique()))
print(len(p3db_tair_ids_valid_unique))

In [ ]:
result = pd.DataFrame(p3db_tair_ids_valid_unique, columns = ['code'])
result.to_csv(out_file_path, sep='\t', index = False)

## vanWijk experimental

In [ ]:
#vanWijk_experimental_data_file_path = path.join(base_path,  "/papers/vanWijk/modif_SupplementalDataSet_2rfinal.csv")
vanWijk_experimental_data_file_path = path.join(base_path,  "data/raw/SupplementalDataSet_2rfinal.zip")

vanWijk_experimental_data = pd.read_csv(vanWijk_experimental_data_file_path, sep = '\t', low_memory=False, skiprows = [0])
print(vanWijk_experimental_data.shape)
#vanWijk_experimental_data.head()

In [ ]:
vanWijk_experimental_genes = vanWijk_experimental_data.gene.unique()
print(len(vanWijk_experimental_genes))
len(vanWijk_experimental_genes) == 8141 #8141 count from nature paper plot

Check all ids are valid:

In [ ]:
vanWijk_experimental_genes_valid = [x for x in vanWijk_experimental_data_phospho.gene if x in all_arabidopsisis_codes]
print(len(vanWijk_experimental_genes_valid))
all([x in all_arabidopsisis_codes for x in vanWijk_experimental_genes_valid])

In [ ]:
vanWijk_experimental_data_phospho = pd.DataFrame(vanWijk_experimental_genes_valid, columns=["gene"])
vanWijk_experimental_data_phospho.head()

In [ ]:
vanWijk_experimental_data_out_file_path = path.join(base_path, 'data/preproc/vanWijk_experimental.csv')
vanWijk_experimental_data_phospho.to_csv(vanWijk_experimental_data_out_file_path, sep = '\t', index = False)

## Mergner experimental

In [ ]:
mergner_experimental_data_file_path = base_path + "/data/raw/41586_2020_2094_MOESM14_ESM.zip"

mergner_experimental_data = pd.read_csv(mergner_experimental_data_file_path, sep = '\t', low_memory=False, skiprows = [0, 1])
print(mergner_experimental_data.shape)
mergner_experimental_data.head()

In [ ]:
mergner_experimental_data_phospho_mask = mergner_experimental_data['AGI code_phosphoprotein'].notna()
print(mergner_experimental_data_phospho_mask.sum())
mergner_experimental_data_phospho_mask.sum() == 8577 #8577 count from nature paper plot

In [ ]:
#phospho_columns = ['AGI code_Araport11', 'AGI code_phosphoprotein']
phospho_columns = ['AGI code_phosphoprotein']
mergner_experimental_data_phospho = mergner_experimental_data.loc[mergner_experimental_data_phospho_mask, phospho_columns]
print(mergner_experimental_data_phospho.shape)
mergner_experimental_data_phospho.head()

Check all ids are valid:

In [ ]:
mergner_experimental_genes_valid = [x for x in mergner_experimental_data_phospho.iloc[:, 0] if x in all_arabidopsisis_codes]
print(len(mergner_experimental_genes_valid))
all([x in all_arabidopsisis_codes for x in mergner_experimental_genes_valid])

In [ ]:
mergner_experimental_data_out_file_path = path.join(base_path, 'data/preproc/mergner_experimental.csv')
mergner_experimental_data_phospho.to_csv(mergner_experimental_data_out_file_path, sep = '\t', index = False)

## Combine datasets

### Sources

In [ ]:
data_baes_path = path.join(base_path, 'data/preproc')
phosphat_data_file_name = "phosphat_experimental_2020.csv.zip"
p3db_data_file_name = "p3db-3.5_experimental.zip"
vanWijk_data_file_name = "vanWijk_experimental.zip"
mergner_data_file_name = "mergner_experimental.zip"


In [ ]:

phosphat_file_path = path.join(data_baes_path, phosphat_data_file_name)
phosphat_data = pd.read_csv(phosphat_file_path, sep = '\t')
phosphat_data.columns = ['code']
print(phosphat_data.shape)

mergner_file_path =  path.join(data_baes_path, mergner_data_file_name)
mergner_data = pd.read_csv(mergner_file_path, sep = '\t')
mergner_data.columns = ['code']
print(mergner_data.shape)

vanWijk_file_path = path.join(data_baes_path, vanWijk_data_file_name)
vanWijk_data = pd.read_csv(vanWijk_file_path, sep = '\t')
vanWijk_data.columns = ['code']
print(vanWijk_data.shape)

p3db_file_path = path.join(data_baes_path, p3db_data_file_name)
p3db_data = pd.read_csv(p3db_file_path, sep = '\t')
p3db_data.columns = ['code']
print(p3db_data.shape)

result_file_path = path.join(base_path, 'data/preproc/experimental_ids.txt')

In [ ]:
all_data_list = [phosphat_data, mergner_data, vanWijk_data, p3db_data]
all_data_raw = pd.concat(all_data_list, axis = 0)
all_data = all_data_raw.code.unique()
print(all_data.shape)
all_data

In [ ]:
result = pd.DataFrame(all_data, columns = ['code'])
result.to_csv(result_file_path, sep='\t', index = False)


phosphat_2020: 8113

p3db: 4215

mergner: 8577

vanWijk: 8078

experimental total: 13137

# BORRAR: 

In [ ]:
musite_phosphat_data_concordance.shape

In [ ]:
phospho_codes_experimental.index = phospho_codes_experimental.code
mergner_experimental_data_phospho.index = mergner_experimental_data_phospho['AGI code_Araport11']
vanWijk_experimental_data_phospho.index = vanWijk_experimental_data_phospho.gene
musite_phosphat_data_concordance.index = musite_phosphat_data_concordance.accession_unified

In [ ]:
prediction_experimental = phospho_codes_experimental.join(musite_phosphat_data_concordance, how="inner")
prediction_experimental.shape

In [ ]:
(8023 - 7149) / 8023

el 10% que dieron fosforilados experimentalmente, no estan en la prediccion

no estan porque dieron discordantes?

no fueron predichos por musite?

comparar que universo es esto en arabidopsis total

## Intersecciones

In [ ]:
result.shape

experimental vs p3db

In [ ]:
len(p3db_data.code)

In [ ]:
int_exp_p3db = [x for x in p3db_data.code.values if x in result.code.values]
len(int_exp_p3db)

In [ ]:
len(mergner_data.code.unique())

In [ ]:
int_exp_mergner = [x for x in mergner_data.code.values if x in result.code.values]
len(int_exp_mergner)

In [ ]:
len(vanWijk_data_clean.code.values)

In [ ]:
int_exp_vanWijk = [x for x in vanWijk_data_clean.code.values if x in result.code.values]
len(int_exp_vanWijk)

In [ ]:
result.code